In [1]:
import os 
import cv2
import sentencepiece as spm
import torch
from tqdm import tqdm


In [2]:
vocab_size = 1207 #토큰 사이즈 

In [3]:
model_name = 'model_'+str(vocab_size) #spm 모델 명

In [4]:
model_type = 'bpe' #토큰화 알고리즘 bpe or unigram

In [5]:
fdname = 'train'+model_type+str(vocab_size)
print(f'fdname {fdname}')

fdname trainbpe1207


In [6]:
fname = '/home/aiv-gpu-019/labels2.txt' #토큰화 알고리즘을 학습시킬 label 파일

In [7]:
f= open(fname, 'r')

for i in range(5):
    print(f.readline().strip())
f.close()


회사 사람들이 다들 나를 피하는 것 같아 우울해
뼈에 필요한 영양소는 우유에서 찾는 거지
내일 첫 배낚시를 하러 가는데 혹시 날씨가 안 좋으면 취소가 되려나
부모님은 내가 미술대학에 진학하는 걸 원하지 않으셔
애견 카페에서 나는 제일 싼 음료를 시키고 우리 강아지는 나보다 비싼 간식을 시켜줬지


In [8]:
if os.path.isdir(fdname) ==False:
    os.mkdir(fdname)

In [9]:
%cd {fdname}

/home/aiv-gpu-019/trainbpe1207


/home/aiv-gpu-019/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [10]:


# SentencePiece 모델을 사용자 정의 어휘로 학습
spm.SentencePieceTrainer.train(
  f"--input={fname} --model_prefix={model_name} --vocab_size={str(vocab_size)}"+
    f" --model_type={model_type}" +
    " --max_sentence_length=999999" + # 문장 최대 길이 (너무 길면 에러발생)
    " --pad_id=1 --pad_piece=<pad>" + # pad (0)
    " --unk_id=2 --unk_piece=<unk>" + # unknown (1)
    " --bos_id=3 --bos_piece=<sos>" + # begin of sequence (2)
    " --eos_id=4 --eos_piece=<eos>" +  # end of sequence (3)
    " --user_defined_symbols=<sep>,<cls>,<mask>") # 사용자 정의 토큰


sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=/home/aiv-gpu-019/labels2.txt --model_prefix=model_1207 --vocab_size=1207 --model_type=bpe --max_sentence_length=999999 --pad_id=1 --pad_piece=<pad> --unk_id=2 --unk_piece=<unk> --bos_id=3 --bos_piece=<sos> --eos_id=4 --eos_piece=<eos> --user_defined_symbols=<sep>,<cls>,<mask>
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: /home/aiv-gpu-019/labels2.txt
  input_format: 
  model_prefix: model_1207
  model_type: BPE
  vocab_size: 1207
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 999999
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 

In [11]:
%cat {model_name}.vocab | head -n 30

<sep>	0
<pad>	0
<unk>	0
<sos>	0
<eos>	0
<cls>	0
<mask>	0
는데	-0
▁나	-1
▁있	-2
▁이	-3
▁아	-4
▁하	-5
▁가	-6
▁내	-7
▁사	-8
▁그	-9
에서	-10
▁같	-11
▁다	-12
▁어	-13
▁해	-14
▁너	-15
▁너무	-16
▁많	-17
▁것	-18
▁좋	-19
친구	-20
▁보	-21
▁오	-22


In [12]:
#아래는  모델에서 사용할 vocab 파일 생성을 위함 
sp = spm.SentencePieceProcessor()
sp.Load(f"{model_name}.model")

True

In [13]:
vocab = {i: sp.IdToPiece(i) for i in range(sp.GetPieceSize())}

In [14]:
with open(f"{model_name}_init.txt", "w") as f_out:
    for idx, (_, s) in enumerate(sorted(vocab.items(), key=lambda x: x[0])):
        f_out.write(f"{s} {idx+1}\n")

In [15]:
#파일 학습에 쓰도록 파일명 변경

In [16]:
%mv {model_name}.model unigram5000.model
%mv {model_name}.vocab unigram5000.vocab
%mv {model_name}_init.txt unigram5000_units.txt

In [ ]:
# 모델 토큰 파일로 위치 복사

In [17]:
dstD = '/DATA/temp/auto_avsr/spm/unigram'

In [18]:
%cd ..
%cp -r {fdname} {dstD}

/home/aiv-gpu-019


In [19]:
'''
%cp unigram5000.model {dstD}
%cp unigram5000.vocab {dstD}
%cp unigram5000_units.txt {dstD}
'''

'\n%cp unigram5000.model {dstD}\n%cp unigram5000.vocab {dstD}\n%cp unigram5000_units.txt {dstD}\n'